In [ ]:
!pip install --upgrade progressbar2

  Attempting uninstall: progressbar2
    Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
# Define root directory from Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)
os.chdir("/content/drive/My Drive/trustpilot")
os.listdir()

Mounted at /content/drive
Mounted at /content/drive


['united_states.auto-adjusted_gender.geocoded.json',
 'france.auto-adjusted_gender.NUTS-regions.json',
 'france.json',
 'trustpilot.pt',
 '.ipynb_checkpoints',
 'united_states.json',
 'trustpilot_france.pt',
 'trustpilot_usa.pt']

In [ ]:
# Import Torch
import torch
# Setup device
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

cuda


In [ ]:
import ast

json_path = "france.auto-adjusted_gender.NUTS-regions.json"
h = open(json_path,'r')

data = []

for l in h.readlines():
    data.append(ast.literal_eval(l))

h.close()

In [ ]:
data[0]['reviews'][0]['text'][0]

In [ ]:
import json
with open('france.json', 'w') as json_file:
    json.dump(data, json_file)

In [ ]:
json_path = "united_states.auto-adjusted_gender.geocoded.json"
h = open(json_path,'r')

data = []

for l in h.readlines():
    data.append(ast.literal_eval(l))

h.close()

In [ ]:
import json
with open('united_states.json', 'w') as json_file:
    json.dump(data, json_file)

In [ ]:
import json
  
# Opening JSON file
f = open('france.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

In [ ]:
france_reviews = []
france_targets = []

In [ ]:
from string import punctuation
print(len(data))
count = 0
for i in range(0, len(data)):
  if (data[i]['reviews'][0]['text'] != [] and (''.join([c for c in data[i]['reviews'][0]['text'][0] if c not in punctuation]))!= '' and count < 30000):
    france_reviews.append(data[i]['reviews'][0]['text'][0])
    france_targets.append(int(data[i]['reviews'][0]['rating']))
    count = count + 1

741237


In [ ]:
print(len(france_reviews))
print(len(france_targets))

30000
30000


In [ ]:
# Remove punctuaction
for i in range(0, len(france_reviews)):
  france_reviews[i] = ''.join([c for c in france_reviews[i] if c not in punctuation])

In [ ]:
print(len(france_reviews))
print(len(france_targets))
print(france_reviews[10])

30000
30000
Jai apprécié la simplicité la rapidité et léconomie merci


In [ ]:
x = [w for w in france_reviews[10].split(" ") if len(w) > 0]

In [ ]:
x

['Jai',
 'apprécié',
 'la',
 'simplicité',
 'la',
 'rapidité',
 'et',
 'léconomie',
 'merci']

In [ ]:
count=0
for r in range(0, len(france_reviews)):
  if len(france_reviews[r])<= 0:
    print(r)
    count=count+1

print(count)

0


In [ ]:
# Split words
france_reviews = [[w for w in r.split(" ") if len(w) > 0] for r in france_reviews if len(r) > 0]

In [ ]:
print(len(france_reviews))
print(len(france_targets))
print(france_reviews[10])
print(france_targets[10])

30000
30000
['Jai', 'apprécié', 'la', 'simplicité', 'la', 'rapidité', 'et', 'léconomie', 'merci']
5


In [ ]:
# Print average review length
review_lens = [len(r) for r in france_reviews]
print(sum(review_lens)/len(review_lens))

19.232166666666668


In [ ]:
# Build vocabulary
# Start index = 1, we need 0 to pad sequences
words = list(set([w for r in france_reviews for w in r]))
vocab = {words[i]: i+1 for i in range(len(words))}

In [ ]:
len(vocab)

32966

In [ ]:
# Convert reviews to word indexes
france_reviews = [[vocab[w] for w in r] for r in france_reviews]

In [ ]:
france_reviews[10]

[27305, 32873, 21865, 24785, 21865, 21598, 5016, 31921, 29622]

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import urllib

# Parameters
seq_len = 20
frac_train = 0.8
frac_val = 0.1
batch_size = 64

In [ ]:
# Clip reviews to max seq_len words
france_reviews = [r[:seq_len] for r in france_reviews]

In [ ]:
# Print average review length now
review_lens = [len(r) for r in france_reviews]
print(sum(review_lens)/len(review_lens))

12.9727


In [ ]:
# Pad reviews shorter than seq_len
france_reviews = [[0]*(seq_len - len(r)) + r for r in france_reviews]

In [ ]:
# Print average review length now
review_lens = [len(r) for r in france_reviews]
print(sum(review_lens)/len(review_lens))

20.0


In [ ]:
# Convert reviews to tensor
data = torch.LongTensor(france_reviews)
print(data.size())

torch.Size([30000, 20])


In [ ]:
# Load sentiments: less than 3 stars is negative (0), 4 and 5 are positive (1)   
sentiments = [0 if f <= 3 else 1 for f in france_targets]

print(len(sentiments))

30000


In [ ]:
print(sentiments[10])

1


In [ ]:
# Convert sentiments to tensor
labels = torch.LongTensor(sentiments)
labels.size()

torch.Size([30000])

In [ ]:
# Shuffle dataset
num_data = data.size(0)
shuffle_idx = torch.randperm(num_data)
data = data[shuffle_idx,:]
labels = labels[shuffle_idx]

In [ ]:
# Split training and test
num_train = int(num_data*frac_train)
num_val = int(num_data*frac_val)
num_test = num_data - num_train - num_val
train_data = data[:num_train,:]
train_labels = labels[:num_train]
val_data = data[num_train:num_train+num_val,:]
val_labels = labels[num_train:num_train+num_val]
test_data = data[num_train+num_val:,:]
test_labels = labels[num_train+num_val:]

In [ ]:
# Create datasets
train_dataset = TensorDataset(train_data, train_labels)
val_dataset = TensorDataset(val_data, val_labels)
test_dataset = TensorDataset(test_data, test_labels)

In [ ]:
# Create loaders
loaders = {"train": DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  drop_last=True),
           "val":   DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, drop_last=True),
           "test":  DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, drop_last=True)}

In [ ]:
# Embedding module:
# - size of input vocabulary
# - size of output vector
test_embed = nn.Embedding(5, 10)

In [ ]:
# Example input to embedding: batch 2, sequence length 3
input = torch.LongTensor([[3, 2, 1],[2, 3, 1]])
print(input)

tensor([[3, 2, 1],
        [2, 3, 1]])


In [ ]:
# Print embedding size
test_embed(input).size()

torch.Size([2, 3, 10])

In [ ]:
# Define model
class Model(nn.Module):
    
    def __init__(self, embed_size, rnn_size):
        # Call parent constructor
        super().__init__()
        # Store values
        self.rnn_size = rnn_size
        # Define modules
        self.embedding = nn.Embedding(len(vocab)+1, embed_size)
        self.rnn = nn.LSTMCell(embed_size, rnn_size)
        self.output = nn.Linear(rnn_size, 2)
        
    def forward(self, x):
        # Embed data
        x = self.embedding(x)
        # Initialize state
        c = x.new_zeros(x.shape[0], self.rnn_size)
        h = x.new_zeros(x.shape[0], self.rnn_size)
        # Input is: BxTxF
        # Process each time step
        for t in range(x.shape[1]):
            # Input at time t
            x_t = x[:,t,:]
            # Forward RNN and get new state
            h,c = self.rnn(x_t, (h, c))
        # Classify final state
        x = self.output(h)
        return x

In [ ]:
# Setup device
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Model parameters
embed_size = 1024
rnn_size = 512

In [ ]:
# Create model
model = Model(embed_size, rnn_size)
model = model.to(dev)

In [ ]:
# Test model output
model = model.to(dev)
model.eval()
test_input = train_dataset[0][0].unsqueeze(0).to(dev)
print("Model output size:", model(test_input).size())

Model output size: torch.Size([1, 2])


In [ ]:
# Create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

In [ ]:
import progressbar as pb
# Start training
for epoch in range(150):
    # Initialize accumulators for computing average loss/accuracy
    epoch_loss_sum = {'train': 0, 'val': 0, 'test': 0}
    epoch_loss_cnt = {'train': 0, 'val': 0, 'test': 0}
    epoch_accuracy_sum = {'train': 0, 'val': 0, 'test': 0}
    epoch_accuracy_cnt = {'train': 0, 'val': 0, 'test': 0}
    # Process each split
    for split in ["train", "val", "test"]:
        # Set network mode
        if split == "train":
            model.train()
            torch.set_grad_enabled(True)
            widgets = [
              ' [', pb.Timer(), '] ',
              pb.Bar(),
              ' [', pb.ETA(), '] ', pb.Variable('ta','[Train Acc: {formatted_value}]')
            ]

            progbar = pb.ProgressBar(max_value=len(loaders[split]),widgets=widgets,redirect_stdout=True)

        else:
            model.eval()
            torch.set_grad_enabled(False)
        # Process all data in split
        data_iter = iter(loaders[split])
        data_len = len(loaders[split])
        for i in range(data_len):
            # Read data
            input, target = next(data_iter)
            # Move to device
            input = input.to(dev)
            target = target.to(dev)
            # Forward
            output = model(input)
            loss = F.cross_entropy(output, target)
            # Update loss sum
            epoch_loss_sum[split] += loss.item()
            epoch_loss_cnt[split] += 1
            # Compute accuracy
            pred = output.argmax(1)
            correct = pred.eq(target).sum().item()
            accuracy = correct/input.size(0)
            # Update accuracy sum
            epoch_accuracy_sum[split] += accuracy
            epoch_accuracy_cnt[split] += 1
            if (split=='train'):
              progbar.update(i, ta=accuracy)
            # Backward and optimize
            if split == "train":
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
    # Compute average epoch loss/accuracy
    avg_train_loss = epoch_loss_sum["train"]/epoch_loss_cnt["train"]
    avg_train_accuracy = epoch_accuracy_sum["train"]/epoch_accuracy_cnt["train"]
    avg_val_loss = epoch_loss_sum["val"]/epoch_loss_cnt["val"]
    avg_val_accuracy = epoch_accuracy_sum["val"]/epoch_accuracy_cnt["val"]
    avg_test_loss = epoch_loss_sum["test"]/epoch_loss_cnt["test"]
    avg_test_accuracy = epoch_accuracy_sum["test"]/epoch_accuracy_cnt["test"]
    if (progbar is not None):
          progbar.finish()
    print(f"Epoch: {epoch+1}, TrL={avg_train_loss:.4f}, TrA={avg_train_accuracy:.4f},",
                            f"VL={avg_val_loss:.4f}, VA={avg_val_accuracy:.4f}, ",
                            f"TeL={avg_test_loss:.4f}, TeA={avg_test_accuracy:.4f}")

 [Elapsed Time: 0:00:25] |###############| [Time:  0:00:25] [Train Acc:  0.922]


Epoch: 1, TrL=0.2836, TrA=0.8932, VL=0.2525, VA=0.8971,  TeL=0.2697, TeA=0.8916


 [Elapsed Time: 0:00:25] |###############| [Time:  0:00:25] [Train Acc:  0.953]


Epoch: 2, TrL=0.2325, TrA=0.9063, VL=0.2571, VA=0.9046,  TeL=0.2718, TeA=0.8961


 [Elapsed Time: 0:00:25] |###############| [Time:  0:00:25] [Train Acc:  0.906]


...

Epoch: 148, TrL=0.0279, TrA=0.9895, VL=0.5502, VA=0.8815,  TeL=0.5852, TeA=0.8784


 [Elapsed Time: 0:00:25] |###############| [Time:  0:00:25] [Train Acc:  0.984]


Epoch: 149, TrL=0.0296, TrA=0.9892, VL=0.4954, VA=0.8702,  TeL=0.5541, TeA=0.8621


 [Elapsed Time: 0:00:25] |###############| [Time:  0:00:25] [Train Acc:    1.0]


Epoch: 150, TrL=0.0286, TrA=0.9890, VL=0.5240, VA=0.8879,  TeL=0.5557, TeA=0.8804


In [ ]:
# Test review
#1 stars
#review = "Beaucoup de déception sur la qualité des produits malgré une descrition accrocheuse Je ne recommanderai pas votre maison à mon entourage"
#5 stars
#review = "Jai apprécié la simplicité la rapidité et léconomie merci"
review = "service client pire de l histoire"
review = [vocab[w] for w in review.split(" ")]
review = [0]*(seq_len - len(review)) + review
review = torch.LongTensor(review).unsqueeze(0).to(dev)
# Process review
model.eval()
output = model(review)
probabilities = F.softmax(output,1)
_,pred = output.max(1)
pred = pred[0]
print(("positive" if pred == 1 else "negative") + f" ({100*probabilities[0,pred]:.1f}%)")

negative (56.7%)


In [ ]:
#save model
model_save_name = 'trustpilot_france.pt'
path = F"/content/drive/My Drive/trustpilot/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
#load model
model_name = 'trustpilot_france.pt'
path = F"/content/drive/My Drive/trustpilot/{model_name}" 
model.load_state_dict(torch.load(path))


<All keys matched successfully>